In [1]:
import pandas as pd
data = pd.read_csv("../Data/gpascore.csv")
data.head()

,admit,gre,gpa,rank
0,0,380.0,3.21,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.33,4


admit : 합격여부 1 합격, 0 불합격       
gre : 영어 성적     
gpa : 학점      
rank : 지원한 대학원 수준       
// kaggle에 있다.

### Preprocessing (전처리)

In [2]:
# 결측치 처리
data.isnull().sum()

admit    0
gre      1
gpa      0
rank     0
dtype: int64

In [3]:
# 결측치 데이터 삭제
data.dropna(inplace= True)

# 결측치 데이터를 다른 값으로 대체하는 경우
# data.fillna(100)

In [4]:
# 처리 결과 확인
data.isnull().sum()

admit    0
gre      0
gpa      0
rank     0
dtype: int64

In [8]:
# rank의 종류 파악
sorted(data['rank'].unique())

[1, 2, 3, 4]

In [9]:
# 영어 점수의 최소값과 최대값
print('최대값 :' , data['gre'].max())
print('최소값 :' , data['gre'].min())

최대값 : 800.0
최소값 : 220.0


In [10]:
# Training Data
x = data[['gre' , 'gpa','rank']]
x.head()

,gre,gpa,rank
0,380.0,3.21,3
1,660.0,3.67,3
2,800.0,4.00,1
3,640.0,3.19,4
4,520.0,2.33,4


In [11]:
# Target Data
y = data['admit']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: admit, dtype: int64

In [14]:
# 갯수
data.describe()
x.count()

gre     425
gpa     425
rank    425
dtype: int64

---
### Deep Learning Model 만들기

In [15]:
from tensorflow import keras

In [16]:
model = keras.Sequential()
# 1번째 Hiddne Layer 만들기 : 256 노드 (보통 2의 배수로 사용, 임의의 갯수 (Hyper Parameter))
# activation Function이 들어가야한다. activation = sigmoid , tanh(하이퍼탄젠트) , relu , softmax , leakyRelu
model.add(keras.layers.Dense(256, activation = 'tanh', input_shape = (3,))) # 은닉층
model.add(keras.layers.Dense(1, activation = 'sigmoid')) # 출력층

2022-07-19 17:36:42.693322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# 손실함수
# 들어 오는 값이 실수 이다
model.compile(optimizer = 'adam', loss='binary_crossentropy' , metrics = 'accuracy')
# 옴션 설명 --------------------------------
# optimizer : learning rate를 알맞게 조정해줌
# - adam 많이 사용, 점프해서 지나가면 다시 돌아오지 않음 
# - adagrad 아담에서 발전, 점프했다가 다시 돌아오기도 함
# - adadelta 
# - rmsprop
# - sgd

# loss 는 손실함수
# - binary_crossentropy : (이진분류) 분류 및 확률 예측

# metrics : 모댈의 평가 기준

In [21]:
# 학습 시키기
import numpy as np
# 딥러닝은 판다스 취급도 안함 넘파이로 변환해줘야 한다.
model.fit(np.array(x) , np.array(y) , epochs= 500 ) # verbose = 0 옵션 설정해주면 실행결과를 줄즐이 안보여준다 

Epoch 1/500
14/14 [==============================] - 0s 1ms/step - loss: 0.5538 - accuracy: 0.7247
Epoch 2/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5726 - accuracy: 0.6988
Epoch 3/500
14/14 [==============================] - 0s 1ms/step - loss: 0.5672 - accuracy: 0.7012
Epoch 4/500
14/14 [==============================] - 0s 1ms/step - loss: 0.5743 - accuracy: 0.7012
Epoch 5/500
14/14 [==============================] - 0s 3ms/step - loss: 0.5707 - accuracy: 0.6941
Epoch 6/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5619 - accuracy: 0.7106
Epoch 7/500
14/14 [==============================] - 0s 1ms/step - loss: 0.5795 - accuracy: 0.6776
Epoch 8/500
14/14 [==============================] - 0s 1ms/step - loss: 0.5637 - accuracy: 0.7176
Epoch 9/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5627 - accuracy: 0.7294
Epoch 10/500
14/14 [==============================] - 0s 2ms/step - loss: 0.5586 - accuracy: 0.7129
Epoch 11/

In [22]:
# 예측하기
model.predict(
    [
        [700 , 3.7 , 1],
        [700 , 3.7 , 2],
        [700 , 3.7 , 3],
        [700 , 3.7 , 4],
    ]
)
# 1,2,3등급 80퍼 4등급 대학 79퍼로 갈수 있음

1/1 [==============================] - 0s 85ms/step


array([[0.8317194 ],
       [0.8207161 ],
       [0.80797905],
       [0.79330367]], dtype=float32)

In [23]:
# 예측하기
model.predict(
    [
        [400 , 2.2 , 1],
        [400 , 2.2 , 2],
        [400 , 2.2 , 3],
        [400 , 2.2 , 4],
    ]
)
# 1~4등급 대학원 다 못갈것이다 

1/1 [==============================] - 0s 29ms/step


array([[0.02438987],
       [0.02141145],
       [0.01889636],
       [0.01677708]], dtype=float32)